In [306]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial.distance import cdist

In [307]:
def load(image_dir, folder):
    path = os.path.join(image_dir, folder)
    files = [f for f in os.listdir(path) if f.endswith('.pgm')]
    labels = np.zeros(len(files), dtype=int)
    images = np.zeros((len(files), 29 * 24))

    for idx, file in enumerate(files):
        image = np.asarray(Image.open(os.path.join(path, file)).resize((24, 29))).reshape(1, -1)
        images[idx, :] = image
        labels[idx] = int(file[7:9])

    return images, labels

In [308]:
X_train, y_train = load('/Users/jolene/Documents/NYCU/Machine_Learning/HW7/Yale_Face_Database', 'Training')
X_test, y_test = load('/Users/jolene/Documents/NYCU/Machine_Learning/HW7/Yale_Face_Database', 'Testing')

In [ ]:
def plot_w(W, method, kernel_f, n_components=25):
    plt.figure(figsize=(6, 8))
    for i in range(n_components):
        plt.subplot(5, 5, i+1)
        plt.axis('off')
        plt.imshow(W[:, i].reshape(29, 24), cmap="gray")
    # plt.show()
    plt.savefig(f"/Users/jolene/Documents/NYCU/Machine_Learning/HW7/Eigenfaces/transform_{method}_{kernel_f}.jpg", format='JPEG')
    plt.close()
    
def plot_reconstruct(re_img, ori_img, method, kernel_f):
    plt.figure(figsize=(6, 8))
    for i in range(10):
        plt.subplot(5, 4, i*2+1)
        plt.axis('off')
        plt.imshow(re_img[i, :].reshape(29, 24), cmap='gray')
        plt.subplot(5, 4, i*2+2)
        plt.axis('off')
        plt.imshow(ori_img[i].reshape(-1, 1).reshape(29, 24), cmap='gray')
    # plt.show()
    plt.savefig(f"/Users/jolene/Documents/NYCU/Machine_Learning/HW7/Eigenfaces/reconstruct_{method}_{kernel_f}.jpg", format='JPEG')
    plt.close()

In [310]:
class Utils:
    @staticmethod
    def eigen_decomposition(S):
        e_values, e_vectors = np.linalg.eig(S)
        idx = np.argsort(e_values)[::-1][:25]
        e_values = e_values[idx]
        e_vectors = e_vectors[:, idx]
        
        pos_idx = np.where(e_values > 0)[0]
        e_values = e_values[pos_idx].real
        e_vectors = e_vectors[:, pos_idx].real
        
        return e_values, e_vectors
    
    @staticmethod
    def kernel(x, kernel_f=None):
        if kernel_f == None:
            return np.cov(x, bias=True)   
          
        elif kernel_f == 'linear':
            K = x @ x.T
        elif kernel_f == 'poly':
            K = (x @ x.T + 1e-5)**3
        elif kernel_f == 'RBF':
            K = np.exp(-1e-8*cdist(x, x, 'sqeuclidean'))
        
        N = x.shape[0]
        one_N = np.ones((N, N)) / N
        Kc = K - one_N@K - K@one_N + one_N@K@one_N
        return Kc

In [ ]:
class Embedding(Utils):
    def __init__(self, X, y, kernel_f=None):
        self.X = X
        self.y = y
        self.kernel_f = kernel_f
        self.X_mean = np.mean(self.X, axis=0)
        self.X_center = self.X - self.X_mean

    def PCA(self):
        S = self.kernel(self.X.T, self.kernel_f) #(696, 696)
    
        _, e_vectors = self.eigen_decomposition(S)
        transform = e_vectors # (696, 25)
        W = self.X_center @ e_vectors # (135, 25)
        z = W.T @ self.X_center # (25, 696)
        reconstruct = W @ z + self.X_mean.reshape(1, -1) # (135, 696)

        return transform, W, z, reconstruct

    def LDA(self, pca_z):
        mu = np.mean(pca_z, axis=1)
        N = pca_z.shape[1]
        Sw, Sb = np.zeros((N, N)), np.zeros((N, N))

        labels = np.unique(self.y)
        for i in range(len(labels)):
            z_j = self.X[self.y == i + 1]
            mu_j = np.mean(z_j, axis=0, keepdims=True).T

            Sw_j = np.cov(z_j.T, bias=True) 
            Sw += Sw_j
            
            n_j = z_j.shape[1]
            Sb += n_j * (mu_j - mu) @ (mu_j - mu).T     

        S = np.linalg.pinv(Sw) @ Sb # (696, 696)
        _, e_vectors = self.eigen_decomposition(S)
        transform = e_vectors # (696, 25)

        W = self.X_center @ e_vectors # (135, 25)
        z = W.T @ self.X_center # (25, 696)
        reconstruct = W @ z + self.X_mean.reshape(1, -1)
        
        return transform, W, z, reconstruct    
    
    def test(self, X_test, y_test, method, k=15):
        test_center = X_test - self.X_mean
        
        if method == 'PCA':
            transform, W, z, _ = self.PCA()
        elif method == 'LDA':
            pca_transform, pca_W, pca_z, _ = self.PCA()
            transform, W, z, _ = self.LDA(pca_W, pca_z)
            
        W_test = test_center @ transform # (30, 25)
        # print(z_test.shape, z.shape)
        # z_test = W.T @ test_center.T @ test_center # (25, 696)
        dist_matrix = cdist(W_test, W, metric='sqeuclidean') # (30, 135)
        
        preds = []
        for i in range(dist_matrix.shape[0]):
            k_idx = np.argsort(dist_matrix[i])[:k]
            
            k_labels = self.y[k_idx]
            pred = np.argmax(np.bincount(k_labels))
            preds.append(pred)

        preds = np.array(preds)
        gts = np.sum(preds == y_test)
        acc = gts / X_test.shape[0] 
        loss =  (X_test.shape[0] - gts) / X_test.shape[0] 
        no_gts = X_test.shape[0] - gts
        
        print(f'===== {method} {self.kernel_f} Kernel =====') 
        print(f'Accuracy: {acc:.2%}')
        print(f'Correct samples: {gts}')
        print(f'Loss: {loss:.2%}')
        print(f'Wrong samples: {no_gts}')


In [312]:
kernel_f = [None, 'linear', 'poly', 'RBF']
for f in kernel_f:      
    embedding = Embedding(X_train, y_train, kernel_f=f)
    PCA_transform, PCA_W, PCA_z, PCA_re = embedding.PCA()
    embedding.test(X_test, y_test, method='PCA')
    plot_w(PCA_transform, kernel_f=f, method='PCA')
    plot_reconstruct(PCA_re, X_train, kernel_f=f, method='PCA')

    LDA_transform, LDA_W, LDA_x, LDA_re = embedding.LDA(PCA_W, PCA_z)
    embedding.test(X_test, y_test, method='LDA')
    plot_w(LDA_transform, kernel_f=f, method='LDA')
    plot_reconstruct(LDA_re, X_train, kernel_f=f, method='LDA')
    
    print('\n')

===== PCA None Kernel =====
Accuracy: 80.00%
Correct samples: 24
Loss: 20.00%
Wrong samples: 6
===== LDA None Kernel =====
Accuracy: 96.67%
Correct samples: 29
Loss: 3.33%
Wrong samples: 1


===== PCA linear Kernel =====
Accuracy: 86.67%
Correct samples: 26
Loss: 13.33%
Wrong samples: 4
===== LDA linear Kernel =====
Accuracy: 96.67%
Correct samples: 29
Loss: 3.33%
Wrong samples: 1


===== PCA poly Kernel =====
Accuracy: 76.67%
Correct samples: 23
Loss: 23.33%
Wrong samples: 7
===== LDA poly Kernel =====
Accuracy: 96.67%
Correct samples: 29
Loss: 3.33%
Wrong samples: 1


===== PCA RBF Kernel =====
Accuracy: 86.67%
Correct samples: 26
Loss: 13.33%
Wrong samples: 4
===== LDA RBF Kernel =====
Accuracy: 96.67%
Correct samples: 29
Loss: 3.33%
Wrong samples: 1


